In [1]:
#Empirical MSE for changing environment
#Uses new f_ave calculation weighting all times equally to new p(t) one.
from scipy.stats import t
from scipy.stats import norm;
import matplotlib.pyplot as plt
from PIL import Image
import random;
import numpy;
import scipy.integrate as integrate;
import scipy.special as special;

# Load Path Data

In [2]:
pathMeanTimes = [];
pathStdOfTimes = [];
pathDistances = [];
percentOfTimeInEachRegion = [];
P_I = [];
P_p = [];
regionConcent = [120,110,80,200,230,100,150];

#1801 Fry Street Falcon Heights, MN 55113 to 163 Bridge Street, Saint Paul, MN 55126 

#via Snelling Ave and Highway 96W. #T = 16min. D = 8.9 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Turn left at the 2nd cross street onto Snelling Ave
#8 min (4.9 mi)
#Follow Hamline Ave N and Hwy 96 W to Bridge St in Shoreview
#7 min (3.8 mi)
pathMeanTimes.append(960);
pathStdOfTimes.append(60);
pathDistances.append(8.9);
P_I.append(0.2);
P_p.append(.25);
percentOfTimeInEachRegion.append([.33,.12,.3,0,0,.25,0]);

#via MN 36-E. T=15min. D=10.2 mi
#Get on MN-36 E in Roseville from Snelling Ave
#4 min (1.3 mi)
#Continue on MN-36 E. Take Exit 113 to Rice St. Take exit 45 from I-694 W/US-10 W
#8 min (6.7 mi)
#Follow Rice St to Bridge St
#4 min (2.2 mi)
pathMeanTimes.append(900);
pathStdOfTimes.append(45);
pathDistances.append(10.2);
P_I.append(0.3);
P_p.append(.20);
percentOfTimeInEachRegion.append([0,.18,0,.07,.12,.32,.31]);

#via Snelling Ave and County Rd F. T = 17min. D = 8.4 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Turn left at the 2nd cross street onto Snelling Ave
#8 min (4.9 mi)
#Follow County Rd F West and Snail Lake Blvd to Bridge St in Shoreview
#8 min (3.3 mi)
pathMeanTimes.append(1020);
pathStdOfTimes.append(70);
pathDistances.append(8.4);
P_I.append(0.3);
P_p.append(.4);
percentOfTimeInEachRegion.append([.23,.22,.3,0,0,.25,0]);

#via Victorian T=19min. D = 8.6 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Take Victoria St N to Snail Lake Rd in Shoreview
#16 min (7.6 mi)
#Turn right onto Snail Lake Rd
#2 min (0.6 mi)
#Turn left onto Rice St
#22 s (0.2 mi)
#Turn left onto Bridge St
#18 s (167 ft)
pathMeanTimes.append(1140);
pathStdOfTimes.append(75);
pathDistances.append(8.6);
P_I.append(0.1);
P_p.append(.05);
percentOfTimeInEachRegion.append([.07,.22,.17,.19,0,.35,0]);

#via MN 36 and Rice ST. T = 17.5 min D =8.9 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Take MN-36 E and Rice St to Bridge St
#16 min (8.7 mi)
#Turn left onto Bridge St
#18 s (167 ft)
pathMeanTimes.append(1050);
pathStdOfTimes.append(65);
pathDistances.append(8.9);
P_I.append(0.1);
P_p.append(.1);
percentOfTimeInEachRegion.append([0,.18,0,.07,.11,.33,.31]);

print("Path Mean Times in seconds: ");
print(pathMeanTimes);
print("\nPath Standard Deviation of Times in seconds: ");
print(pathStdOfTimes);
print("\nPath Distances in miles:");
print(pathDistances);
print("\nPath Percent in each Region: ");
print(percentOfTimeInEachRegion);
print("\nP_I(x):");
print(P_I);
print("\nP_p(x):")
print(P_p);

Path Mean Times in seconds: 
[960, 900, 1020, 1140, 1050]

Path Standard Deviation of Times in seconds: 
[60, 45, 70, 75, 65]

Path Distances in miles:
[8.9, 10.2, 8.4, 8.6, 8.9]

Path Percent in each Region: 
[[0.33, 0.12, 0.3, 0, 0, 0.25, 0], [0, 0.18, 0, 0.07, 0.12, 0.32, 0.31], [0.23, 0.22, 0.3, 0, 0, 0.25, 0], [0.07, 0.22, 0.17, 0.19, 0, 0.35, 0], [0, 0.18, 0, 0.07, 0.11, 0.33, 0.31]]

P_I(x):
[0.2, 0.3, 0.3, 0.1, 0.1]

P_p(x):
[0.25, 0.2, 0.4, 0.05, 0.1]


# Simulation

In [3]:
#initial values
#runs = 21;
updateInterval = 1;

def trainingsession(runs):#returns [px,tdf,tloc,tscale] for 'runs' number of test runs
    N = runs;

    pi_m = [];
    pi_s = [];
    numberOfTimesRouteTaken = [0,0,0,0,0];
    routeTravelTimes=[[],[],[],[],[]]; #stores only history since last update
    allRouteTravelTimes=[[],[],[],[],[]]; #stores all the history

    m = [0,0,0,0,0];
    c=[0,0,0,0,0];
    tbar=[0,0,0,0,0];
    sumtijsquared = [0,0,0,0,0]; 
    v=[1,1,1,1,1];
    a = [0.5,0.5,0.5,0.5,0.5];
    b = [0.1,0.1,0.1,0.1,0.1];   

#tdistribution params
    tloc = [950,930,1000,1090,1100]; #mean
    tscale = [50, 60, 65, 80, 55]; #"stdedv"
    tdf = [1,1,1,1,1]; #degrees of freedom

    ptx = [0,0,0,0,0]; # probability of time t given route x
    px = [0,0,0,0,0]; # probability of route x
    pbxt = [0,0,0,0,0]; # probability of route x given time t




    for x in range(0,N):
    
        #Pick a random path based on P_I
        randomNumber = random.uniform(0,1);
        pathLimit=0;
        for r in range(0,len(P_I)):
            pathLimit = pathLimit+P_I[r];
            if(randomNumber<pathLimit):
                randomPath = r;
                break;
        #print("\nPath: "+str(randomPath));
    
        #Find a random time that it took for that path based on normal distribution for that path
        pathUserTime = random.normalvariate(pathMeanTimes[randomPath],pathStdOfTimes[randomPath]);
        #print("User Time in Seconds: "+str(pathUserTime));
        routeTravelTimes[randomPath].append(pathUserTime);
        allRouteTravelTimes[randomPath].append(pathUserTime);
        numberOfTimesRouteTaken[randomPath] = numberOfTimesRouteTaken[randomPath] + 1;
        c[randomPath] = c[randomPath]+1;
        tbar[randomPath] = (tbar[randomPath]*(len(routeTravelTimes[randomPath])-1)+pathUserTime)/len(routeTravelTimes[randomPath]);
        sumtijsquared[randomPath] = sumtijsquared[randomPath] + pow(pathUserTime,2);
    
        #Calculate actual time in each weather region
        actualTimeInEachRegion = [];    
        for r in range(0,len(percentOfTimeInEachRegion[randomPath])):
            actualTimeInEachRegion.append(percentOfTimeInEachRegion[randomPath][r]*pathUserTime);
        
        #print("Time in each region: "+str(actualTimeInEachRegion));
    
        #print("tbar: " + str(tbar[randomPath]));
        #print("c: " + str(c[randomPath]) );
        #print("sumtijsquared: " + str(sumtijsquared[randomPath]) );
    
        if(c[randomPath]%updateInterval==0):
            #Update prior hyperparameters
            oldm = m[randomPath];
            oldv = v[randomPath];
            olda = a[randomPath];
            oldb = b[randomPath];

            v[randomPath] = oldv/(1+c[randomPath]*oldv);
            m[randomPath] = (oldm/oldv+c[randomPath]*tbar[randomPath])*v[randomPath]        
            a[randomPath] = olda+c[randomPath]/2;

            b[randomPath] = oldb + (1/2)*(pow(oldm,2)/oldv+sumtijsquared[randomPath]-pow(m[randomPath],2)/v[randomPath]);
        
            c[randomPath] = 0; 
            tbar[randomPath] = 0;
            sumtijsquared[randomPath] = 0;
            routeTravelTimes[randomPath] = [];
        
            #print("UPDATED HYPER PARAMS \nm: "+str(m[randomPath])+
            #"\nv: " + str(v[randomPath]) + 
            #"\na: " + str(a[randomPath]) +
            #"\nb: " + str(b[randomPath]));
        
            #Update Tdistribution params
    for x in range(0,len(numberOfTimesRouteTaken)):    
        tloc[x] = m[x];
        tscale[x] = numpy.sqrt(b[x]*(1+v[x])/(a[x])); # should be under sqrt?
        tdf[x] = 2*a[x]; #2a?
        #tdf[x] = a[x];
        #print(str(tdf[x]));
        
#print("\n Sum of numberOfTimesRouteTaken = "+str(sum(numberOfTimesRouteTaken))); # just checking my understanding.
#print("\n N = "+str(N)); # just checking my understanding.
        
#p(t|x)
#print("\nProbabilities for t ="+str(pathUserTime)+" (generated from route number "+str(randomPath)+")."); #using only the most recent travel time, pathUserTime for now.
#for x in range(0,len(numberOfTimesRouteTaken)):
#    print("ROUTE "+str(x)+"df = "+str(tdf[x]));
#    arr = numpy.array(allRouteTravelTimes[x]);
    #std = numpy.std(arr, axis=0); #used for delta t in p(t) calc.
#    std = 20;
#    tinterval = (1/20)*std;
    #ptx[x]=tdf[x]/5;
    #ptx[x] = t.cdf(pathUserTime+tinterval, 8, 1, 4)-t.cdf(pathUserTime-tinterval,  8, 1, 4);
#    ptx[x] = t.cdf(pathUserTime+tinterval, tdf[x], tloc[x], tscale[x])-t.cdf(pathUserTime-tinterval,  tdf[x], tloc[x], tscale[x]);
#    print("p(t|x): "+str(ptx[x]));
#    px[x] = numberOfTimesRouteTaken[x]/sum(numberOfTimesRouteTaken);
#    print("p(x): "+str(px[x]));

#sumOfProbabilities = 0;
#for x in range(0,len(numberOfTimesRouteTaken)):    
#    sumOfProbabilities = sumOfProbabilities + ptx[x]*px[x]; #this is p(t)

#for x in range(0,len(numberOfTimesRouteTaken)):     
#    pbxt[x] = (ptx[x]*px[x])/sumOfProbabilities;
#    print("Route " + str(x) +" pb(x|t): "+str(pbxt[x]));





    #print("\nProbabilities for t ="+str(pathUserTime)+" (generated from route number "+str(randomPath)+")."); #using only the most recent travel time, pathUserTime for now.
    for x in range(0,len(numberOfTimesRouteTaken)):
        #print("ROUTE "+str(x)+"df = "+str(tdf[x]));
        #arr = numpy.array(allRouteTravelTimes[x]);
        #std = numpy.std(arr, axis=0); #used for delta t in p(t) calc.
        #std = 20;
        #tinterval = (1/20)*std;
        ptx[x] = t.pdf(pathUserTime, tdf[x], tloc[x], tscale[x]);#this is p(t|x)
        #print("p(t|x): "+str(ptx[x]));
        px[x] = numberOfTimesRouteTaken[x]/sum(numberOfTimesRouteTaken); #this is p(x)
        #print("p(x): "+str(px[x]));

    sumOfProbabilities = 0;
    for x in range(0,len(numberOfTimesRouteTaken)):    
        sumOfProbabilities = sumOfProbabilities + ptx[x]*px[x]; #this is p(t)

    for x in range(0,len(numberOfTimesRouteTaken)):     
        pbxt[x] = (ptx[x]*px[x])/sumOfProbabilities; #this is p(x|t)
        #print("Route " + str(x) +" pb(x|t): "+str(pbxt[x]));
        




    #print("\n Sum of prob x given t = "+str(sum(pbxt))); # just checking my understanding.

    estimateInEachRegion = []
    #Estimated exposure
    for j in range(0,len(actualTimeInEachRegion)):
        estimateForThisRegion = 0;
        for i in range(0,len(numberOfTimesRouteTaken)): #len(numberOfTimesRouteTaken)= number of candidate paths
            estimateForThisRegion = estimateForThisRegion+percentOfTimeInEachRegion[i][j]*pathUserTime*pbxt[i];
        estimateInEachRegion.append(estimateForThisRegion);

    #print("Estimated Exposure: " + str(estimateInEachRegion));
    #print("g(t) ="+str(sum(estimateInEachRegion)));
    return [px,tdf,tloc,tscale,runs]

print(str(trainingsession(10)))


[[0.2, 0.3, 0.4, 0.1, 0.0], [3.0, 4.0, 5.0, 2.0, 1.0], [646.2897365234671, 662.3229018279661, 794.6772984761365, 563.1666634964495, 0], [529.60540353204681, 428.4641441974361, 437.86666898631012, 689.73559159331819, 0.63245553203367588], 10]


In [4]:
def training(numberOfPaths,pathAndTime,trainingDays,mInit):#minit = list of initialization for m
    m = mInit;
    c=[];
    tbarTotal=[];
    tbar = [];
    sumtijsquared = [];
    numberOfTimesRouteTaken = [];
    v=[];
    a = [];
    b = [];
    px = [];
    tloc = [];
    tscale = [];
    tdf = [];
    for path in range(0,numberOfPaths):
        c.append(0);
        v.append(1);
        a.append(0.01);
        b.append(0.01);
        c.append(0);
        tbarTotal.append(0);
        tbar.append(0);
        sumtijsquared.append(0);
        
        #numberOfTimesRouteTaken.append(0);
    for day in range(0,trainingDays):
        path = pathAndTime[day][0];
        c[path] = c[path]+1;
        sumtijsquared[path] = sumtijsquared[path] + pathAndTime[day][1]**2;
        tbarTotal[path] = tbarTotal[path] + pathAndTime[day][1];
    for path in range(0,numberOfPaths):
        if c[path]==0:
            tbar[path]=0;
        else:
            tbar[path] = tbarTotal[path]/c[path];
        

    for path in range(0,numberOfPaths):
        oldm = m[path];
        oldv = v[path];
        olda = a[path];
        oldb = b[path];
    
    
        v[path] = oldv/(1+c[path]*oldv);
        m[path] = (oldm/oldv+c[path]*tbar[path])*v[path];
        a[path] = olda+c[path]/2;
        b[path] = oldb + (1/2)*(pow(oldm,2)/oldv+sumtijsquared[path]-pow(m[path],2)/v[path]);
        tloc.append(m[path]);
        tscale.append(numpy.sqrt(b[path]*(1+v[path])/(a[path])));
        tdf.append(2*a[path]);
        px.append(c[path]/trainingDays);
        
    return [px,tdf,tloc,tscale,trainingDays];

In [5]:
def pt(t,pind,indPathMeans,indPathStds): #calculates true probability of time t for:
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    for i in range(0,len(pind)):
        total = total + norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    return total


In [6]:
#determining f(x,t) and average value


low = -1000
high = 3000
#These are the limits of integraton for time in the average value calculations for f_ave and for the MSE.
#They must be carefully set so that you integrate over all significantly likely times, but not
#so wide that you get division by 0 for times for which p(t) is indistinguishable from 0.



def fexposure(path, time,concens):#path = path, time = time, concens = list of concentrations in each region
    #requires global specification of percent of time in each reagion, percentOfTimeInEachRegion
    #which is a list of lists.  There is one list for each path and that list is the list
    #of percents of times that path spends in each region.  This list must be of length len(concens)
    actualCITI = [];
    actualTimeInEachRegion = [];    
    for r in range(0,len(percentOfTimeInEachRegion[path])):
        actualTimeInEachRegion.append(percentOfTimeInEachRegion[path][r]*time);
    for regionNum in range(0,len(actualTimeInEachRegion)): #sum of atualCITI = f(x,t)
        actualCITI.append(concens[regionNum]*actualTimeInEachRegion[regionNum]);
    return sum(actualCITI);



def avef(baseconcentrations,pind,indPathMeans,indPathStds): #average f_environ(x,t) over all routes 
    #times and environments
    #baseconcentrations = list of base pollutant concentrations in each cell
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    newvec = []
    for i in range (0,len(P_I)):
        pct_conc = sum(j[0] * j[1] for j in zip(percentOfTimeInEachRegion[i], baseconcentrations))
        newvec.append(pct_conc*pind[i]*pathMeanTimes[i]);
    return sum(newvec);



def avefold(baseconcentrations,pind,indPathMeans,indPathStds): #Old, but accurate
    #average f_environ(x,t) over all routes 
    #times and environments
    #baseconcentrations = list of base pollutant concentrations in each cell
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    #timeNormalization = integrate.quad(pt,low,high,args=(pind,indPathMeans,indPathStds))[0];
    timeNormalization = 1;
    for i in range(0,len(pind)):
        def f(t):
            #return fexposure(i,t,(baseconcentrations))*pt(t,pind,indPathMeans,indPathStds);
            return fexposure(i,t,(baseconcentrations))*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
        total = total + integrate.quad(f,low,high,args=())[0];
    return total/timeNormalization;


In [7]:
#Generating path and time list

def pathAndTimeGen(days):
    indPathAndTime = [];
    #Pick a random path based on P_I
    for i in range(0,days):
        randomNumber = random.uniform(0,1);
        pathLimit=0;
        for r in range(0,len(P_I)):
            pathLimit = pathLimit+P_I[r];
            if(randomNumber<pathLimit):
                randomPath = r;
                break;
        #Find a random time that it took for that path based on normal distribution for that path
        pathUserTime = random.normalvariate(pathMeanTimes[randomPath],pathStdOfTimes[randomPath]);
        indPathAndTime.append([randomPath,pathUserTime]);
    return indPathAndTime;

def environGen(regions,base,variation,days):#makes a list of environmental conditions
    #for days number of days
    #regions = number of Voronoi cells
    #base = base concentration list, a list of length = regions
    #variation = plus/minus range of variation in the values
    environment = [];
    for i in range(0,days):
        dailyConditions = [];
        for region in range(0,regions):
            dailyConditions.append(base[region]+random.uniform(-variation,variation));
        environment.append(dailyConditions);
    return environment;


In [8]:
#TESTING##




#print(str(term1[0]));
#print(str(term2[0]));
#print("ave f(x,t) = "+str(avef()));
#print("\n");
#print("Theoretically minimal sqrt of MSE = "+str(sqrtmse));
#print("Theoretically minimal relative sqrt of MSE = "+str(relativemse));
#print("\n");
#print("Theoretically minimal MSE ="+str(mse));
#print("Theoretically mimimal relative MSE ="+str(mse/avef()))
#trainedParam = trainingsession(10);

#print(str(px));
#print(str(P_I));
#print("\n")



#########
#EMPERICAL MSE START

def probtBel(time,trainedParam):#thIs is updated belief p_B(time)
    result = 0.0;
    for x in range(0,len(trainedParam[0])):
        result = result + t.pdf(time, trainedParam[1][x], trainedParam[2][x], trainedParam[3][x])*trainedParam[0][x];
    return result

#print(str(probt(900)));

def probxgiventBel(x,time,trainedParam):#this is updted belief p_B(x|time)
    return t.pdf(time, trainedParam[1][x], trainedParam[2][x], trainedParam[3][x])*trainedParam[0][x]/probtBel(time,trainedParam)

#print("2 given 900 = "+str(probxgivent(2,900)))


def gest(time,trainedParam, concens):#this is g(time)
    total = 0;
    for x in range(0,len(trainedParam[0])):
        total = total + fexposure(x,time,concens)*probxgiventBel(x,time,trainedParam)
    return total

#print("g(960) = "+str(gest(960)))





def empAveSquareError(trainedParam,pathAndTime, environment, startDay, periodLength):#Returns
    #empirically observed average square error
    #trainedParam = individual's hyperparameters after most current training
    #pathAndtime = list of [path,time] observations for the user on which we train and test
    #environment = list of lists, each of which is daily conditions in the Voronoi cells
    #startDay = first day on which to start testing
    #periodLength = length (in days) of this testing perios
    squaresum = 0;
    dayContribution = 0;
    for day in range(0,periodLength):
        path = pathAndTime[startDay-1][0];
        time = pathAndTime[startDay-1][1];
        concens = environment[startDay-1];
        dailyContribution = (fexposure(path,time,concens)-gest(time,trainedParam,concens))**2
        squaresum = squaresum + dailyContribution;
    return squaresum/periodLength;


userData = pathAndTimeGen(150);
environment = environGen(7,[1.1,1.8,1.4,0.9,1.6,1.0,1.8],0.3,150);

    


    
#M = 100;
#testMSE = testAveSquareError(M,trainedParam);
#relativeTestMSE = numpy.sqrt(testMSE)/avef()
#print("ave f(x,t) = "+str(avef()));
#print("\n");
#print("Theoretically minimal sqrt of MSE = "+str(sqrtmse));
#print("Theoretically minimal relative sqrt of MSE = "+str(relativemse));
#print("\n");
#print("Theoretically minimal MSE ="+str(mse));
#print("Theoretically mimimal relative MSE ="+str(mse/avef()))

#print(str(M)+" run test observed average MSE = "+str(testMSE));
#print(str(trainedParam[4])+" run training session.")
#print(str(M)+" run test observed sqrt of average MSE = "+str(numpy.sqrt(testMSE)))
#print(str(M)+" run test observed relative sqrt average MSE = "+str(relativeTestMSE))
#print("\n")
#print(str(M)+" run relative distance from optimal = "+str((relativeTestMSE-relativemse)/relativemse))


In [9]:
#Multiple Empirical MSE calculations.


#########
#THEORETICALLY MINIMAL MSE FOR CHANGING ENVIRONMENT BEGIN



low = 200
high = 2650
#These are the limits of integraton for time in the average value calculations for f_ave and for the MSE.
#They must be carefully set so that you integrate over all significantly likely times, but not
#so wide that you get division by 0 for times for which p(t) is indistinguishable from 0.

#########
#THEORETICALLY MINIMAL MSE START
def pt(t,pind,indPathMeans,indPathStds): #calculates true probability of time t for:
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    for i in range(0,len(pind)):
        total = total + norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    return total



def avef(baseconcentrations,pind,indPathMeans,indPathStds): #average f_environ(x,t) over all routes 
    #times and environments
    #baseconcentrations = list of base pollutant concentrations in each cell
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    timeNormalization = integrate.quad(pt,low,high,args=(pind,indPathMeans,indPathStds))[0];
    for i in range(0,len(pind)):
        def f(t):
            return fexposure(i,t,(baseconcentrations))*pt(t,pind,indPathMeans,indPathStds);
        total = total + integrate.quad(f,low,high,args=())[0];
    return total/(len(pind)*timeNormalization);


def integrand(t, concens,pind,indPathMeans,indPathStds):#integrand in the MSE calculation
    #concens = list of pollutant concentrations in the cells
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    term1 = 0;
    term2sum = 0;
    term2 = 0;
    for i in range(0,len(pathMeanTimes)):
        term1 = term1 + ((fexposure(i,t,concens))**2)*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    for i in range(0,len(pathMeanTimes)):
        term2sum = term2sum + fexposure(i,t,concens)*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    term2 = (term2sum**2)/pt(t,pind,indPathMeans,indPathStds);
    return term1 - term2;


def theoreticalmse(M,baseconc,confidencepct,pind,indPathMeans,indPathStds):#computes confidence interval
    #for relative MSE with:
    #M = number of runs (trials)
    #baseconc = base concentration list
    #confidencepct  = confidence level in percent
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    sqrunErAve = 0.0; #running average mean squared error
    frunAve =  0.0; # running average average value of f_epsilon(x,t)
    sqErtotal = 0.0; # total running square error
    relErrorList = [];
    errorList = [];
    averagefValue = avef(baseconc,pind,indPathMeans,indPathStds);
    for trial in range(0,M):
        concens = [0,0,0,0,0,0,0];
        for region in range(len(baseconc)):
            concens[region] = baseconc[region];#+random.uniform(-0.9,0.9);
        mse = integrate.quad(integrand,low,high,args=(concens,pind,indPathMeans,indPathStds))[0];
        #integrate from low to high for about 6 standard deviations above and below mean times
        errorList.append(mse);
        relErrorList.append(mse/(averagefValue**2));
        print(str(trial), end="");
    #
    std = numpy.std(relErrorList,ddof=1); #sample stdev of observed relative error
    tcv = t.ppf((100-confidencepct)/200,df = M-1,loc = 0, scale = 1); #critical value for our confidence
    mseest = numpy.mean(relErrorList);
    mseSqrtest = numpy.sqrt(mseest);
    confidence = [mseest - (-tcv*std)/numpy.sqrt(M), mseest +(-tcv*std)/numpy.sqrt(M)];
    #
    msePctest = mseest*100;
    mseSqrtPctest = mseSqrtest*100;
    msePctConfidence = [];
    mseSqrtPctConfidence = [];
    for i in range(0,len(confidence)):
        msePctConfidence.append(100*confidence[i]);
        mseSqrtPctConfidence.append(100*numpy.sqrt(confidence[i]));
    
    return [msePctest,msePctConfidence,mseSqrtPctest, mseSqrtPctConfidence,confidencepct];


#########
#THEORETICALLY MINIMAL MSE END


base=[1.1,1.8,1.4,0.9,1.6,1.0,1.8];

theorymse = theoreticalmse(2,base,99,P_I,pathMeanTimes,pathStdOfTimes);


relativemse = theorymse[2];


#runs of empirical MSE





testTestsAbs = [];
testTestsRel = [];
reps = 10;
concentrations=[1.1,1.8,1.4,0.9,1.6,1.0,1.8];
trainedParam = training(5,userData,10,[0,0,0,0,0]);
for reps in range(0,reps):
#    trainedParam = trainingsession(10);
    M = 10;
    testMSE = empAveSquareError(trainedParam,userData,environment,M*reps,10);
    relativeTestMSE = numpy.sqrt(testMSE)/avef(concentrations,P_I,pathMeanTimes,pathStdOfTimes);
    testTestsRel.append(round(100*(relativeTestMSE-relativemse)/relativemse,2));
    testTestsAbs.append(round(100*relativeTestMSE,2));
    
    
print("\n");
print("\n");
print("ave f(x,t) = "+str(avef(concentrations,P_I,pathMeanTimes,pathStdOfTimes)));
print("\n");
#print("Theoretically minimal sqrt of MSE = "+str(sqrtmse));
print("Theoretically minimal relative sqrt of MSE = "+str(relativemse));
print("\n");
    #print("Theoretically minimal MSE ="+str(mse));
    #print("Theoretically mimimal relative MSE ="+str(mse/avef()))

    #print(str(M)+" run test observed average MSE = "+str(testMSE));
print(str(trainedParam[4])+" run training session.")
print("\n");
    #print(str(M)+" run test observed sqrt of average MSE = "+str(numpy.sqrt(testMSE)))
    
print(str(M)+" run test observed relative sqrt average MSE, pct = ")
for i in range(0,reps+1):
    print(str(testTestsAbs[i])+"  ", end="");
print("\n");
#print(str(M)+" run relative distance from optimal, pct = ");
#for i in range(0,reps+1):
#    print(str(testTestsRel[i])+"  ", end="");
        #print(str(M)+" run test observed relative sqrt average MSE, pct = "+"\n"+str(testTestsAbs))
print("\n")
    #print(str(M)+" run relative distance from optimal, pct = "+"\n"+str(testTestsRel));
print("\n")
print("Average rel dist, pct = "+str(sum(testTestsRel)/reps))


    
    
    

01



ave f(x,t) = 1324.1592


Theoretically minimal relative sqrt of MSE = 5.93216459083


10 run training session.


10 run test observed relative sqrt average MSE, pct = 
1.54  7.7  15.85  11.61  7.58  2.98  3.93  7.4  20.39  15.4  





Average rel dist, pct = -109.342222222


In [10]:
print("hello")


hello
